In [3]:
import numpy as np
import pandas as pd
from sqlalchemy import create_engine

# Загрузка файлов, путь в Google Colab всегда "/content/название_файла"
df_olist_customers = pd.read_csv('olist_customers_dataset.csv')
df_olist_sellers = pd.read_csv('olist_sellers_dataset.csv')
df_olist_order_reviews = pd.read_csv('olist_order_reviews_dataset.csv')
df_olist_order_items = pd.read_csv('olist_order_items_dataset.csv')
df_olist_products = pd.read_csv('olist_products_dataset.csv')
df_olist_geolocation = pd.read_csv('olist_geolocation_dataset.csv')
df_product_category_name_translation = pd.read_csv('product_category_name_translation.csv')
df_olist_orders = pd.read_csv('olist_orders_dataset.csv')
df_olist_order_payments = pd.read_csv('olist_order_payments_dataset.csv')

# Проверим первую таблицу
df_olist_customers.head()

# Создаём SQLite базу в памяти
engine = create_engine('sqlite://', echo=False)

# Экспортируем таблицы
df_olist_customers.to_sql("olist_customers", con=engine, index=False, if_exists='replace')
df_olist_sellers.to_sql("olist_sellers", con=engine, index=False, if_exists='replace')
df_olist_order_reviews.to_sql("olist_order_reviews", con=engine, index=False, if_exists='replace')
df_olist_order_items.to_sql("olist_order_items", con=engine, index=False, if_exists='replace')
df_olist_products.to_sql("olist_products", con=engine, index=False, if_exists='replace')
df_olist_geolocation.to_sql("olist_geolocation", con=engine, index=False, if_exists='replace')
df_product_category_name_translation.to_sql("product_category_name_translation", con=engine, index=False, if_exists='replace')
df_olist_orders.to_sql("olist_orders", con=engine, index=False, if_exists='replace')
df_olist_order_payments.to_sql("olist_order_payments", con=engine, index=False, if_exists='replace')

# Проверим данные
df_olist_order_payments.head()


,order_id,payment_sequential,payment_type,payment_installments,payment_value
0,b81ef226f3fe1789b1e8b2acac839d17,1,credit_card,8,99.33
1,a9810da82917af2d9aefd1278f1dcfa0,1,credit_card,1,24.39
2,25e8ea4e93396b6fa0d3dd708e76c1bd,1,credit_card,1,65.71
3,ba78997921bbcdc1373bb41e913ab953,1,credit_card,8,107.78
4,42fdf880ba16b47b59251dd489d4441a,1,credit_card,2,128.45


In [4]:
# Step 2
# Make SQLite file
engine = create_engine('sqlite:///olist_data.db', echo=False)

# Экспортируем таблицы
df_olist_customers.to_sql("olist_customers", con=engine, index=False, if_exists='replace')
df_olist_sellers.to_sql("olist_sellers", con=engine, index=False, if_exists='replace')
df_olist_order_reviews.to_sql("olist_order_reviews", con=engine, index=False, if_exists='replace')
df_olist_order_items.to_sql("olist_order_items", con=engine, index=False, if_exists='replace')
df_olist_products.to_sql("olist_products", con=engine, index=False, if_exists='replace')
df_olist_geolocation.to_sql("olist_geolocation", con=engine, index=False, if_exists='replace')
df_product_category_name_translation.to_sql("product_category_name_translation", con=engine, index=False, if_exists='replace')
df_olist_orders.to_sql("olist_orders", con=engine, index=False, if_exists='replace')
df_olist_order_payments.to_sql("olist_order_payments", con=engine, index=False, if_exists='replace')


sql = '''
SELECT *
FROM olist_customers
LIMIT 5
'''
df_sql = pd.read_sql_query(sql, con=engine)
df_sql.head()


,customer_id,customer_unique_id,customer_zip_code_prefix,customer_city,customer_state
0,06b8999e2fba1a1fbc88172c00ba8bc7,861eff4711a542e4b93843c6dd7febb0,14409,franca,SP
1,18955e83d337fd6b2def6b18a428ac77,290c77bc529b7ac935b93aa66c333dc3,9790,sao bernardo do campo,SP
2,4e7b3e00288586ebd08712fdd0374a03,060e732b5b29e8181a18229c7b0b2b5e,1151,sao paulo,SP
3,b2b6027bc5c5109e529d4dc6358b12c3,259dac757896d24d7702b9acbbff3f3c,8775,mogi das cruzes,SP
4,4f2d8ab171c80ec8364f7c12e35b23ad,345ecd01c38d18a9036ed96c73b8d066,13056,campinas,SP


In [5]:
# Step 3
import pandas as pd

# SQL-запрос
sql = '''
WITH jan_orders AS (
    SELECT order_id
    FROM olist_orders
    WHERE order_purchase_timestamp >= '2018-01-01'
      AND order_purchase_timestamp < '2018-02-01'
),
jan_orders_with_reviews AS (
    SELECT jo.order_id, r.review_score
    FROM jan_orders jo
    JOIN olist_order_reviews r
      ON jo.order_id = r.order_id
)
SELECT
    COUNT(CASE WHEN review_score = 5 THEN 1 END) AS five_star_orders,
    COUNT(*) AS total_orders,
    ROUND(
        100.0 * COUNT(CASE WHEN review_score = 5 THEN 1 END) / COUNT(*),
        2
    ) AS five_star_percentage
FROM jan_orders_with_reviews;
'''

# Выполнение запроса
df_result = pd.read_sql_query(sql, con=engine)

# Просмотр результата
df_result.head()


,five_star_orders,total_orders,five_star_percentage
0,4097,7245,56.55


In [6]:
# Step 4
import pandas as pd

# SQL-запрос: годовой тренд по количеству уникальных покупателей
sql = '''
SELECT
    STRFTIME('%Y', order_purchase_timestamp) AS year,
    COUNT(DISTINCT customer_id) AS unique_customers
FROM
    olist_orders
GROUP BY
    year
ORDER BY
    year;
'''

# Выполнение запроса
df_yearly_trend = pd.read_sql_query(sql, con=engine)

# Просмотр результата
df_yearly_trend.head()


,year,unique_customers
0,2016,329
1,2017,45101
2,2018,54011


In [7]:
# Step 5
import pandas as pd

# SQL-запрос: средняя стоимость заказов по каждому клиенту
sql = '''
WITH order_values AS (
    SELECT
        oi.order_id,
        o.customer_id,
        SUM(oi.price) AS order_total
    FROM
        olist_order_items oi
    JOIN
        olist_orders o
        ON oi.order_id = o.order_id
    GROUP BY
        oi.order_id
)
SELECT
    customer_id,
    ROUND(AVG(order_total), 2) AS avg_order_value
FROM
    order_values
GROUP BY
    customer_id
ORDER BY
    avg_order_value DESC
LIMIT 10;
'''

# Выполнение запроса
df_avg_order_values = pd.read_sql_query(sql, con=engine)

# Просмотр результата
df_avg_order_values.head()


,customer_id,avg_order_value
0,1617b1357756262bfa56ab541c47bc16,13440.0
1,ec5b2ba62e574342386871631fafd3fc,7160.0
2,c6e2731c5b391845f6800c97401a43a9,6735.0
3,f48d464a0baaea338cb25f816991ab1f,6729.0
4,3fd6777bbce08a352fddd04e4a7cc8f6,6499.0


In [8]:
# Task 6
import pandas as pd

# SQL-запрос: топ-5 городов по выручке за 2016–2018
sql = '''
WITH order_totals AS (
    SELECT
        o.order_id,
        c.customer_city,
        SUM(oi.price + oi.freight_value) AS revenue
    FROM
        olist_orders o
    JOIN
        olist_customers c ON o.customer_id = c.customer_id
    JOIN
        olist_order_items oi ON o.order_id = oi.order_id
    WHERE
        o.order_purchase_timestamp >= '2016-01-01'
        AND o.order_purchase_timestamp < '2019-01-01'
    GROUP BY
        o.order_id, c.customer_city
)
SELECT
    customer_city,
    ROUND(SUM(revenue), 2) AS total_revenue
FROM
    order_totals
GROUP BY
    customer_city
ORDER BY
    total_revenue DESC
LIMIT 5;
'''

# Выполнение запроса
df_top_cities = pd.read_sql_query(sql, con=engine)

# Вывод результата
df_top_cities.head()


,customer_city,total_revenue
0,sao paulo,2170227.12
1,rio de janeiro,1154234.02
2,belo horizonte,416733.39
3,brasilia,352305.14
4,curitiba,244739.87


In [9]:
# Task 7
import pandas as pd

# SQL-запрос: выручка по штатам покупателей с 2016 по 2018
sql = '''
WITH order_totals AS (
    SELECT
        o.order_id,
        c.customer_state,
        SUM(oi.price + oi.freight_value) AS revenue
    FROM
        olist_orders o
    JOIN olist_customers c
        ON o.customer_id = c.customer_id
    JOIN olist_order_items oi
        ON o.order_id = oi.order_id
    WHERE
        o.order_purchase_timestamp >= '2016-01-01'
        AND o.order_purchase_timestamp < '2019-01-01'
    GROUP BY
        o.order_id, c.customer_state
)
SELECT
    customer_state,
    ROUND(SUM(revenue), 2) AS total_revenue
FROM
    order_totals
GROUP BY
    customer_state
ORDER BY
    total_revenue DESC;
'''

# Выполняем SQL-запрос
df_state_revenue = pd.read_sql_query(sql, con=engine)

# Вывод результата
df_state_revenue.head()


,customer_state,total_revenue
0,SP,5921678.12
1,RJ,2129681.98
2,MG,1856161.49
3,RS,885826.76
4,PR,800935.44


In [10]:
# Task 8
import pandas as pd

# SQL-запрос: топ-продавцы по продажам, выручке, количеству клиентов и 5-звёздочным оценкам
sql = '''
WITH seller_sales AS (
    SELECT
        seller_id,
        COUNT(*) AS total_items_sold,
        SUM(price + freight_value) AS total_revenue,
        COUNT(DISTINCT o.customer_id) AS unique_customers
    FROM
        olist_order_items oi
    JOIN olist_orders o
        ON oi.order_id = o.order_id
    GROUP BY
        seller_id
),
seller_5stars AS (
    SELECT
        oi.seller_id,
        COUNT(*) AS five_star_reviews
    FROM
        olist_order_reviews r
    JOIN olist_order_items oi
        ON r.order_id = oi.order_id
    WHERE
        r.review_score = 5
    GROUP BY
        oi.seller_id
)
SELECT
    s.seller_id,
    s.total_items_sold,
    ROUND(s.total_revenue, 2) AS total_revenue,
    s.unique_customers,
    COALESCE(f.five_star_reviews, 0) AS five_star_reviews
FROM
    seller_sales s
LEFT JOIN
    seller_5stars f
    ON s.seller_id = f.seller_id
ORDER BY
    total_items_sold DESC
LIMIT 10;
'''

# Выполнение запроса
df_top_sellers = pd.read_sql_query(sql, con=engine)

# Вывод результата
df_top_sellers.head()


,seller_id,total_items_sold,total_revenue,unique_customers,five_star_reviews
0,6560211a19b47992c3666cc44a7e94c0,2033,151265.77,1854,1024
1,4a3ca9315b744ce9f8e9374361493884,1987,235539.96,1806,947
2,1f50f920176fa81dab994f9023523100,1931,142104.98,1404,1096
3,cc419e0650a3c5ba77189a1882b7556a,1775,129957.41,1706,1053
4,da8622b14eb17ae2831f4ac5b9dab84a,1551,185192.32,1314,893


In [11]:
# Task 9
import pandas as pd

# SQL-запрос: delivery success rate по каждому штату
sql = '''
WITH order_status_by_state AS (
    SELECT
        c.customer_state,
        COUNT(*) AS total_orders,
        COUNT(CASE
            WHEN o.order_status = 'delivered'
              AND o.order_delivered_customer_date IS NOT NULL
            THEN 1
        END) AS successful_deliveries
    FROM
        olist_orders o
    JOIN
        olist_customers c
        ON o.customer_id = c.customer_id
    GROUP BY
        c.customer_state
)
SELECT
    customer_state,
    total_orders,
    successful_deliveries,
    ROUND(
        100.0 * successful_deliveries / total_orders,
        2
    ) AS delivery_success_rate
FROM
    order_status_by_state
ORDER BY
    delivery_success_rate DESC;
'''

# Выполнение запроса
df_delivery_success = pd.read_sql_query(sql, con=engine)

# Просмотр результата
df_delivery_success.head()


,customer_state,total_orders,successful_deliveries,delivery_success_rate
0,AC,81,80,98.77
1,AP,68,67,98.53
2,ES,2033,1995,98.13
3,MS,715,701,98.04
4,AM,148,145,97.97


In [12]:
# Task 10
import pandas as pd

# SQL-запрос: наиболее популярная форма оплаты по категориям товаров
sql = '''
WITH category_payments AS (
    SELECT
        p.product_category_name,
        pay.payment_type,
        COUNT(*) AS payment_count
    FROM
        olist_order_items oi
    JOIN olist_products p
        ON oi.product_id = p.product_id
    JOIN olist_order_payments pay
        ON oi.order_id = pay.order_id
    GROUP BY
        p.product_category_name, pay.payment_type
),
ranked_payments AS (
    SELECT
        product_category_name,
        payment_type,
        payment_count,
        RANK() OVER (
            PARTITION BY product_category_name
            ORDER BY payment_count DESC
        ) AS rnk
    FROM
        category_payments
)
SELECT
    product_category_name,
    payment_type AS preferred_payment_type,
    payment_count
FROM
    ranked_payments
WHERE
    rnk = 1
ORDER BY
    payment_count DESC;
'''

# Выполнение запроса
df_payment_preferences = pd.read_sql_query(sql, con=engine)

# Просмотр результата
df_payment_preferences.head()


,product_category_name,preferred_payment_type,payment_count
0,cama_mesa_banho,credit_card,8959
1,beleza_saude,credit_card,7566
2,esporte_lazer,credit_card,6635
3,moveis_decoracao,credit_card,6379
4,informatica_acessorios,credit_card,5436


In [18]:
# Task 11
import pandas as pd

# Повторно загружаем нужные DataFrame'ы
customers = pd.read_csv("olist_customers_dataset.csv")
sellers = pd.read_csv("olist_sellers_dataset.csv")
orders = pd.read_csv("olist_orders_dataset.csv")
order_items = pd.read_csv("olist_order_items_dataset.csv")
geolocation = pd.read_csv("olist_geolocation_dataset.csv")

# Возьмем средние координаты для каждого zip_code_prefix
geo_avg = geolocation.groupby("geolocation_zip_code_prefix")[["geolocation_lat", "geolocation_lng"]].mean().reset_index()

# Добавим координаты покупателей
customers_geo = customers.merge(geo_avg, left_on="customer_zip_code_prefix", right_on="geolocation_zip_code_prefix", how="left")
customers_geo = customers_geo.rename(columns={"geolocation_lat": "customer_lat", "geolocation_lng": "customer_lng"})

# Добавим координаты продавцов
sellers_geo = sellers.merge(geo_avg, left_on="seller_zip_code_prefix", right_on="geolocation_zip_code_prefix", how="left")
sellers_geo = sellers_geo.rename(columns={"geolocation_lat": "seller_lat", "geolocation_lng": "seller_lng"})

# Соберем таблицу заказов: order_id, customer_id, seller_id
order_data = orders[["order_id", "customer_id"]].merge(
    order_items[["order_id", "seller_id"]], on="order_id", how="inner"
)

# Присоединим координаты
order_data = order_data.merge(customers_geo[["customer_id", "customer_lat", "customer_lng"]], on="customer_id", how="left")
order_data = order_data.merge(sellers_geo[["seller_id", "seller_lat", "seller_lng"]], on="seller_id", how="left")

# Удалим пропуски
order_data = order_data.dropna(subset=["customer_lat", "customer_lng", "seller_lat", "seller_lng"])

# Функция расчета расстояния (Haversine)
from math import radians, cos, sin, asin, sqrt

def haversine(row):
    lon1, lat1, lon2, lat2 = map(radians, [row['customer_lng'], row['customer_lat'], row['seller_lng'], row['seller_lat']])
    dlon = lon2 - lon1
    dlat = lat2 - lat1
    a = sin(dlat/2)**2 + cos(lat1) * cos(lat2) * sin(dlon/2)**2
    c = 2 * asin(sqrt(a))
    km = 6371 * c
    return km

# Рассчитаем расстояние
order_data["distance_km"] = order_data.apply(haversine, axis=1)

# Покажем первые строки
order_data_filtered = order_data[["order_id", "seller_id", "customer_id", "distance_km"]].head(10)
order_data_filtered


,order_id,seller_id,customer_id,distance_km
0,e481f51cbdc54678b7cc49136f2d6af7,3504c0cb71d7fa48d967e0e4c94d59d9,9ef432eb6251297304e76186b10a928d,18.576110
1,53cdb2fc8bc7dce0b6741e2150273451,289cdb325fb7e7f891c38608bf9e0962,b0830fb4747a6c6d20dea0b8c802d7ef,851.495069
2,47770eb9100c2d0c44946d9cf07ec65d,4869f7a5dfa277a7dca6462dcf3b52b2,41ce2a54c0b03bf3443c3d931a367089,514.410666
3,949d5b44dbf5de918fe9c16f97b45f8a,66922902710d126a0e7d26b0e3805106,f88197465ea7920adcdbec7375364d82,1822.226336
4,ad21c59c0840e6cb83a9ceb5573f8159,2c9e548be18521d1c43cde1c582c6de8,8ab97904e6daea8866dbdbc4fb7aad2c,29.676625
5,a4591c265e18cb1dcee52889e2d8acc3,8581055ce74af1daba164fdbd55a40de,503740e9ca751ccdda7ba28e9ab8f608,411.394362
6,136cce7faa42fdb2cefd53fdc79a6098,dc8798cbf453b7e0f98745e396cc5616,ed0271e0b7da060a393796590e7b737a,913.640074
7,6514b8ad8028c9f2cc2374ded245783f,16090f2ca825584b5a147ab24aa30c86,9bdf08b4b3b52b5526ff42d37d47f222,322.277082
8,76c6e866289321a7c93b82b54852dc33,63b9ae557efed31d1f7687917d248a8d,f54a9f0e6b351c431402b8461ea51999,869.946044
9,e69bfb5eb88e0ed6a785585b27e16dbf,7c67e1448b00f6e969d365cea6b010ab,31ad1d1b63eb9962463f764d4e6e0c9d,112.263374
